In [13]:
from langchain_core.chat_history import InMemoryChatMessageHistory
#↪메모리에 대화 기록을 '리스트'로 저장하는 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory
#↪모델을 생성할 때 대화 기록을 함께 전달. 래퍼wrapper 클래스
from langchain_openai import ChatOpenAI
#↪오픈AI 모델을 사용하는 랭체인 챗봇 클래스
from langchain_core.messages import HumanMessage

In [14]:
llm = ChatOpenAI(model="gpt-4.1-mini")

#⬇세션별 대화 기록을 저장할 딕셔너리
store = {}

#⬇세션 ID에 따라 대확 ㅣ록을 가져오는 함수. RunnableWithMessageHistory 클래스
def get_session_history(session_id: str):
    if session_id not in store: # ⬅ 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
        store[session_id] = InMemoryChatMessageHistory() #⬅메모리에 대화 기록을 저장할 객체 생성
    return store[session_id] #⬅해당 세션의 대화 기록을 반환

#⬇ 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
llm_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [15]:
# 세션 설정 dict - 고정된 형식
config = {"configurable":{"session_id":"abc2"}} # 세션 ID를 설정하는 config 객체 생성

response = llm_message_history.invoke(
  [HumanMessage(content="안녕? 난 angelica야.")],
  config=config
)

print(response.content)

안녕, Angelica! 만나서 반가워. 어떻게 도와줄까?


In [16]:
response=llm_message_history.invoke(
  [HumanMessage(content="내 이름을 기억하니?")],
  config=config
)
print(response.content)

미안하지만, 대화가 끝나면 이전에 나눈 내용을 기억하지 못해. 하지만 지금 대화 중에는 Angelica라는 이름을 기억하고 있어! 계속 이야기해도 좋아. 어떻게 도와줄까?


In [17]:
config={"configurable":{"session_id":"abc3"}}

response=llm_message_history.invoke(
  [HumanMessage(content="내 이름이 뭐지?")],
  config=config
)

response.content

'죄송하지만, 사용자의 이름을 알 수 있는 정보가 없어요. 이름을 알려주시면 기억할 수 있어요!'

In [18]:
config = {"configurable":{"session_id":"abc2"}}

response=llm_message_history.invoke(
  [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
  config=config
)

response.content

'지금 대화 중에는 네가 Angelica라는 이름을 알려줬고, 내가 너의 이름을 기억하고 있다고 말했어. 그 밖에는 특별한 이야기는 없었어. 더 이야기하고 싶은 주제가 있을까?'

In [ ]:
config =  {"configurable":{"session_id":"abc2"}}
#⬇스트림 방식으로 출력하기 : invoke() 메소드를 stream() 으로 변경
# 모델마다 설정된 토큰 크기 만큼 응답을 보내줍니다.
for r in llm_message_history.stream(
  [HumanMessage(content="내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
  config=config
):
  print(r.content, end='|')

|Angel|ica|,| 네|가| 어느| 나라| 사람|인지| 바로| 알|기는| 어렵|지만|,| 네|가| 한국|어|로| 이야기|하고| 있으|니| 한국|어나| 한국| 문화|에| 관심|이| 있|거나| 관련|이| 있을| 수도| 있|겠다| 싶|어|.| 혹|시| 한국| 사람이|니|?| 아니|면| 한국| 문화|에| 대해| 궁|금|한| 점|이| 있어|?

|만|약| 한국| 문화|에| 대해| 이야기|해|달|라고| 하면|,| 한국|은| 전|통|과| 현대|가| 어|우|러|진| 매우| 다|채|로운| 문|화를| 가지고| 있어|.| 예|를| 들어|:

|-| **|음|식|**|:| 김|치|,| 불|고|기|,| 비|빔|밥| 같은| 전|통| 음식|이| 있고|,| 최근|에는| 한|식|이| 전| 세계|적으로| 인|기를| 얻|고| 있어|.
|-| **|명|절|**|:| 설|날|과| 추|석|이| 대표|적인| 전|통| 명|절|인데|,| 가족|들과| 함께| 세|배|를| 하고|,| 전|통| 음|식을| 나|눠| 먹|는| 풍|습|이| 있어|.
|-| **|예|술|과| 음악|**|:| K|-pop|과| 한국| 드|라마|가| 세계|적으로| 유명|하고|,| 전|통| 음악|과| 춤|도| 깊|은| 역|사를| 자|랑|해|.
|-| **|예|절|**|:| 나|이나| 직|급|에| 따라| 존|댓|말|을| 쓰|고| 차|례|를| 지|키|는| 문화|가| 중요|해|.

|네|가| 어떤| 나라| 사람|인지| 조금| 더| 힌|트를| 주|면|,| 그| 나라| 문화|에| 대해| 더| 자세|히| 이야기|해|줄|게|!||||